In [ ]:
import os
import shutil

from arc.common import read_yaml_file

from t3 import T3
from tests.common import almost_equal, run_minimal
from t3.simulate.cantera_IDT import CanteraIDT, get_t_and_p_lists, get_idt_per_phi_p_condition, plot_idt_vs_temperature
from t3.utils.fix_cantera import fix_cantera

In [ ]:
# %matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
#Run this block only if you want: Chemkin to Cantera YAML Conversion
import os
import numpy as np

import cantera as ct
from cantera import ck2yaml

print(ct.__version__)

ck_mech_path = "/home/nelly/Code/scripts/IDT/data/2BTHF_2017/iteration_1/RMG/cantera/25DMF_COMPLETE.MECH"
ck_thermo_path="/home/nelly/Code/scripts/IDT/data/2BTHF_2017/iteration_1/RMG/cantera/25DMF.THERM"
ck_trans_path="/home/nelly/Code/scripts/IDT/data/2BTHF_2017/iteration_1/RMG/cantera/25DMF.TRAN"

base_path = os.path.dirname(ck_mech_path)
ct_yml_path = os.path.join(base_path, 'chem.yaml')

if os.path.isfile(ck_mech_path):
    print("ok")

parser = ck2yaml.Parser()
parser.convert_mech(ck_mech_path, thermo_file=ck_thermo_path, out_name=ct_yml_path)

In [ ]:
#Edit your IDT project name and iteration nmber
project_name = "2BTHF_2017"
iteration_num = 1

SIM_DIR = os.path.join(os.getcwd(),f'data/{project_name}')
FIG_DIR_IDT = os.path.join(SIM_DIR,f'iteration_{iteration_num}/Figures/')

In [ ]:
#Load T3 input file of your project into a dictionairy
yaml_file_path = os.path.join(SIM_DIR,'input.yml')
input_dict = read_yaml_file(yaml_file_path)

#print the T3 input file dictionairy:
for k,v in input_dict.items():
    print(k,v)

#create a t3 object from your T3 run:
rmg_args = input_dict["rmg"]
t3_args = input_dict["t3"]
qm_args = input_dict["qm"]

t3 = T3(project=project_name,
        t3=t3_args,
        rmg=rmg_args,
        qm=qm_args,
        )

t3.iteration = iteration_num

In [ ]:
t3.project_directory = SIM_DIR
t3.set_paths()
t3.rmg['reactors'] = [{'type': 'gas batch constant T P',
                        'T': [700, 1990], 'P': 20.0,
                        'termination_rate_ratio': 0.01},
                        ]
t3.rmg['species'] = [{'label': 'BTHFX2', 'smiles': 'O1CCCC1(CCCC)', 'concentration': 0, 'role': 'fuel',
                        'equivalence_ratios': [0.5, 1.0, 1.5]},
                        {'label': 'O2', 'smiles': '[O][O]', 'concentration': 0, 'role': 'oxygen'},
                        {'label': 'N2', 'smiles': 'N#N', 'concentration': 0, 'role': 'nitrogen'}]
ct_adapter = CanteraIDT(t3=t3.t3,
                        rmg=t3.rmg,
                        paths=t3.paths,
                        logger=t3.logger,
                        atol=t3.rmg['model']['atol'],
                        rtol=t3.rmg['model']['rtol'],
                        )
exp_data = None

In [ ]:
###Edit your experimental data:
exp_1000_per_T = [0.7478984054806,
                      0.982714059690052,
                      1.39124627841621,
                      1.30022103757639,
                      1.38041467656884,
                      1.29550459904133,
                      1.20775864151308,
                      1.16069874064187,
                      1.14682844593277,
                      1.08077729503059,
                      0.949667616334283,
                      1.41548827267966,
                      1.38523341182989,
                      1.32275132275132,
                      1.20560364574542,
                      1.01067270375162,
                      0.877701125212843,
                      0.851999216160721,]
#The simulation plots IDT [sec] vs. 1000/T [K]
exp_idt = [0.000007,
                  0.000676,
                  0.002251,
                  0.001755,
                  0.002957,
                  0.001831,
                  0.001407,
                  0.001467,
                  0.001471,
                  0.001195,
                  0.00042,
                  0.003031,
                  0.002224,
                  0.001458,
                  0.001409,
                  0.000686,
                  0.000165,
                  0.000124,]

exp_data =(exp_1000_per_T, exp_idt)

In [ ]:
ct_adapter.simulate()
plot_idt_vs_temperature(idt_dict=ct_adapter.idt_dict,figs_path=FIG_DIR_IDT,exp_data = exp_data)

In [ ]:
# t3.rmg['reactors'] = [{'type': 'gas batch constant T P',
#                         'T': [700, 1990], 'P': [1, 50],
#                         'termination_rate_ratio': 0.01},
#                         ]
# t3.rmg['species'] = [{'label': '2BF', 'smiles': 'CCCCC1=CC=CO1', 'concentration': 0, 'role': 'fuel',
#                         'equivalence_ratios': [0.5, 1.5]},
#                         {'label': 'O2', 'smiles': '[O][O]', 'concentration': 0, 'role': 'oxygen'},
#                         {'label': 'N2', 'smiles': 'N#N', 'concentration': 0, 'role': 'nitrogen'}]
